In [2]:
import time
import json
import numpy as np
import pandas as pd
from pandas import json_normalize

In [3]:
def load_df(csv_path, nrows = None):
    json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(csv_path,
                     #converters are dict of functions for converting values in certain columns. Keys can either be integers or column labels.
                     #json.loads() method can be used to parse a valid JSON string and convert it into a Python Dictionary.
                     #It is mainly used for deserializing native string, byte, or byte array which consists of JSON data into Python Dictionary.
                     converters = {col: json.loads for col in json_cols},                                                                         
                         dtype = {'fullVisitorId': 'str'}, # Important!!
                         nrows = nrows)
    for col in json_cols:
        # for each column, flatten data frame such that the values of a single col are spread in different cols
        # This will use subcol as names of flat_col.columns
        flat_col = json_normalize(df[col])
        # Name the columns in this flatten data frame as col.subcol for tracability
        flat_col.columns = [f"{col}.{subcol}" for subcol in flat_col.columns]
        # Drop the json_col and instead add the new flat_col
        df = df.drop(col, axis = 1).merge(flat_col, right_index = True, left_index = True)
    return df


csv_test_path = './test_v2.csv'
test = load_df(csv_test_path, nrows = None)


In [80]:
import matplotlib.pyplot as plt
import json
inp = test.loc[2]
inp = inp['hits']
type(test['geoNetwork.country'])
cnt=0
sum=0
lowCountries=[]
hitWithCountry = {}
cnt = test['geoNetwork.country'].value_counts(sort=True)

for i in range(0, cnt.size):
    sum = sum + cnt[i]

#Identify the Non Operating Countries

for i in range(0, cnt.size):
    #print(f"Country Name: {cnt.index[i]} Frequency: {(cnt[i]/sum)*100}%" )
    if((cnt[i]/sum)*100 < 0.01):
        lowCountries.append(cnt.index[i]) 
    

inputCountries = test.at[0,'geoNetwork.country']

for indx in range(test.index.start, test.index.stop):
    country = test.at[indx, 'geoNetwork.country']
    oneIndex = (test.at[indx,'hits'])
    hitsInfo = ast.literal_eval(oneIndex)
    #print(hitsInfo)
    if country in lowCountries:
        hit = test.at[indx, 'totals.hits']
        if int(hit) > 15:
            print(f"Index Number : {indx} Anomalous traffic Info: {country} Total Hits : {int(hit)}")
            for ind in range(0, len(hitsInfo)):
                print(f"Hit Number: {hitsInfo[ind]['hitNumber']} Visited Page: {hitsInfo[ind]['appInfo']['screenName']} Keyword: {test.at[indx, 'trafficSource.keyword']}")
        #print(f"Country Name: {country} --> Total hits: {hit}")
        if country in hitWithCountry.keys():
            hitWithCountry[country] += 1
        else:
            hitWithCountry[country]=1

#newArr = test['hits'].to_numpy()


Index Number : 4867 Anomalous traffic Info: Papua New Guinea Total Hits : 47
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/google+redesig

Index Number : 9596 Anomalous traffic Info: Guyana Total Hits : 16
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/home Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/basket.html Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/store-policies/return-policy/home Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/asearch.html Keyword: (not set)
Hit Number: 9 Visited Page: shop.googlemerchandisestore.com/store-policies/

Index Number : 51503 Anomalous traffic Info: Somalia Total Hits : 16
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/signin.html Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/signin.html Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/signin.html Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/registersuccess.html Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/store.html Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/store.html Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 9 Visited Page: shop.googlemerchandisestore.com/basket.html Keyword: (not set)
Hit Number: 10 Visited Page: shop.googlemerchandisest

Index Number : 95934 Anomalous traffic Info: Mauritius Total Hits : 24
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/signin.html Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/registersuccess.html Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/store.html Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/google+redesign/apparel Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/google+redesign/apparel/mens Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/google+redesign/accessories/electronic+accessories Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/google+redesign/accessories/fun Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/google+redesign/accessories/stickers+and+decals/home Keyword: (not set)
Hit Number: 9 Visited Page: shop.googlemerchandisestore.com/goog

Index Number : 141518 Anomalous traffic Info: Barbados Total Hits : 50
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop

...

Index Number : 302684 Anomalous traffic Info: Mauritius Total Hits : 19
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/google+redesign/apparel/mens Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/google+redesign/bags Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/google+redesign/accessories Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/google+redesign/accessories/google+wallet+stand+black Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/google+redesign/office Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/store-policies/shipping-information/home Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/signin.html Keyword: (not set)
Hit Number: 9 Visited Page: shop.googlemerchandisestore.com/re

Index Number : 316947 Anomalous traffic Info: Barbados Total Hits : 23
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/home Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/google+redesign/bags Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/home Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/home Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/store-policies/frequently-asked-questions/home Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 9 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/

Index Number : 324324 Anomalous traffic Info: Bermuda Total Hits : 31
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/google+redesign/lifestyle/fun/malibu+sunglasses.axd Keyword: (not provided)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/google+redesign/lifestyle/fun/malibu+sunglasses.axd Keyword: (not provided)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/basket.html Keyword: (not provided)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/store.html Keyword: (not provided)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/store.html Keyword: (not provided)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware Keyword: (not provided)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/google+redesign/accessories Keyword: (not provided)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/google+redesign/apparel Keyword: (not provided)
Hit Number: 9 Visited Page: shop.googlemer

Index Number : 362081 Anomalous traffic Info: St. Vincent & Grenadines Total Hits : 16
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/google+redesign/shop+by+brand/youtube/quickview Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/goog

Index Number : 395345 Anomalous traffic Info: Mauritius Total Hits : 21
Hit Number: 1 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware Keyword: (not set)
Hit Number: 2 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware Keyword: (not set)
Hit Number: 3 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware/quickview Keyword: (not set)
Hit Number: 4 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware Keyword: (not set)
Hit Number: 5 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware/quickview Keyword: (not set)
Hit Number: 6 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware Keyword: (not set)
Hit Number: 7 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware/quickview Keyword: (not set)
Hit Number: 8 Visited Page: shop.googlemerchandisestore.com/google+redesign/drinkware Keyword: (not set)
Hit Number: 9 Visited Page: shop.googlemerchandisestore.co

In [75]:
import ast
oneIndex = (test.at[0,'hits'])
result = ast.literal_eval(oneIndex)
#print(type(result))
for ind in range(0, len(result)):
    print(f"Hit Number: {result[ind]['hitNumber']} Visited Page: {result[ind]['appInfo']['screenName']} Keyword: {result[ind]['trafficSource.keyword']}")

KeyError: 'trafficSource.keyword'

In [74]:
test['trafficSource.keyword']

0         (not provided)
1              (not set)
2         (not provided)
3              (not set)
4         (not provided)
               ...      
401584    (not provided)
401585    (not provided)
401586    (not provided)
401587    (not provided)
401588         (not set)
Name: trafficSource.keyword, Length: 401589, dtype: object